In [3]:
import os
import pandas as pd
import csv
import urllib3
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

directory = "../Data/London/"
index_file = 0
data_files = pd.DataFrame(columns=["File_Name", "Description"])

# Get all files in the directory

In [4]:
for subdir_1 in os.listdir(directory):
    if not subdir_1.startswith("."):
        print("Directory: " + subdir_1)
        for subdir_2 in os.listdir(directory + subdir_1):
            print("File :" + subdir_2) 
        print()

Directory: Census 2001 Key Statistics 21: Long Term Illness   London Datastore
File :Census 2001 Key Statistics 21: Long Term Illness   London Datastore.csv

Directory: Road Collisions by Severity   London Datastore
File :Road Collisions by Severity   London Datastore.csv

Directory: Life Expectancy at Birth and Age 65 by Ward   London Datastore
File :Life Expectancy at Birth and Age 65 by Ward   London Datastore.csv

Directory: Number of Registered Blind and Partially Sighted People with an Additional Disability by Category, Borough   London Datastore
File :Number of Registered Blind and Partially Sighted People with an Additional Disability by Category, Borough   London Datastore.csv

Directory: Drug Treatment Rates   London Datastore
File :Drug Treatment Rates   London Datastore.csv

Directory: Physically Active Children, Borough   London Datastore
File :Physically Active Children, Borough   London Datastore.csv

Directory: Estimation of Health Impacts of Particulate Pollution in Lo

# Download files from the website

Store the files in folders and create a .csv file with the file names and descriptions extracted from the website


In [7]:
url = 'https://data.london.gov.uk/dataset?format=csv&topics=4199df0d-d454-4373-b710-aeed29098a59'
response = requests.get(url)
soup_main = BeautifulSoup(response.text, "html.parser")

#Get the number of pages
pages_links = soup_main.find_all("li", {"class": "dp-search__pagelink"})
pages = [val.string for val in pages_links]
number_pages = pages[len(pages) - 2]
print("Total number of pages: " + str(number_pages))

for index_page in range(1, int(number_pages) + 1):
    current_url_page = "https://data.london.gov.uk/dataset?format=csv&topics=4199df0d-d454-4373-b710-aeed29098a59" + "&page=" + str(index_page)
    response = requests.get(current_url_page)
    soup_main = BeautifulSoup(response.text, "html.parser")
    search_results = soup_main.find_all("a", {"class": "dp-searchresult__heading-link"})
    print("Current page:" + str(index_page))
    index_file = download_items(search_results, index_file)

Total number of pages: 3
Current page:1


/Users/gabrielrosca98/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (10,11,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/gabrielrosca98/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Current page:2
Current page:3


In [6]:
def download_items(search_results, index_file):
    cleaner = re.compile('<.*?>')
    data = list()
    index = 0
    description_data = list()
    for link in search_results:
        url_specific = "https://data.london.gov.uk"
        dataset_url = url_specific + link['href']
        response = requests.get(dataset_url)
        soup_file = BeautifulSoup(response.text, "html.parser")
        buttons_page = soup_file.find_all("a", {"class": "dp-resource__button"})
        for link_2 in buttons_page:
            url_download = url_specific + link_2['href']
            if(link_2['href'].endswith(".csv")):
                title_file = re.sub(cleaner, '', str(soup_file.title))
                title_file = re.sub(r'[^\x00-\x7F]+',' ', title_file)
                try:
                    os.mkdir("../Data/London/" + title_file)
                except OSError:
                    time.sleep(1)
                data_download = pd.read_csv(url_download, sep=",", encoding='cp1252')  
                data_download.to_csv('../Data/London/' + title_file + "/" + title_file + ".csv" , index=True)
                time.sleep(1)
        description_file = soup_file.find("div", {"class": "dp-dataset__description content"})
        description = ""
        for tag in description_file: 
            cleantext = re.sub(cleaner, '', str(tag))
            description += cleantext
        description_data.append(description)
        data_files.loc[index_file] = [re.sub(cleaner, '', re.sub(r'[^\x00-\x7F]+',' ', str(soup_file.title))), description]
        index_file = index_file + 1
        time.sleep(1)
    return index_file

# Data files indexer

In [10]:
data_files.head()

,File_Name,Description
0,Bariatric incidents attended by LFB – London D...,The London Fire Brigade attends a range of non...
1,Animal rescue incidents attended by LFB – Lond...,The London Fire Brigade attends a range of non...
2,GLA Grants data – London Datastore,The GLA Grants Dataset contains data relating ...
3,Breathe London AQMesh pods – London Datastore,The Breathe London stationary network is made ...
4,Natural Capital Account for London – London Da...,This is the first natural capital account for ...


### Get the word count for each description

In [78]:
data_files['Word_Count'] = data_files['Description'].apply(lambda x: len(str(x).split(" ")))
data_files[["Description",'Word_Count']].head()

,Description,Word_Count
0,The London Fire Brigade attends a range of non...,204
1,The London Fire Brigade attends a range of non...,164
2,The GLA Grants Dataset contains data relating ...,85
3,The Breathe London stationary network is made ...,132
4,This is the first natural capital account for ...,65


### Get the most common words

In [17]:
frequency_words = pd.Series(" ".join(data_files['Description']).split()).value_counts()
print("Most frequent words: ") 
print(frequency_words[:10])

Most frequent words: 
the       108
and        65
of         57
is         43
to         42
for        33
The        30
London     28
data       26
a          21
dtype: int64


### Get the most uncommon words

In [36]:
print(frequency_words[-10:])

decide               1
subsequent           1
Money                1
Unit:                1
(Killed/seriously    1
Portraits            1
points               1
There                1
prepared             1
where                1
dtype: int64


### Text pre processing

In [19]:
import string

words_test = data_files['Description']

### Remove punctuation

In [21]:
words_test = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in words_test]

#### Stemming

In [22]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
print(words_test)

['The GLA Grants Dataset contains data relating to grants which have been awarded by the GLA since 2013 The dataset uses the 360 Giving Standard to ensure the data is clear and accessible This dataset can also be accessed on the 360 giving navigation site GrantNav which allows grantmakers and others to explore how grants are used areas of commonality between grantmakers and gaps that are not reached by grantmakers Potential grantseekers can also see who and what the GLA has funded in the past\xa0', 'The London Fire Brigade attends a range of nonfire incidents which we call special services These special services include assistance to animals that may be trapped or in distress \nWe routinely get asked for information about the number of such incident attended by the London Fire Brigade and this data is published on the London Datastore to assist those who require it\nThe data is provided from January 2009 and isupdated monthly A range of information is supplied for each incident includi

In [23]:
stemmed_words = [" ".join(list(map(lambda x: porter_stemmer.stem(x), words.split()))) for words in words_test]
for sentence in stemmed_words:
    print(sentence)
    print()

the gla grant dataset contain data relat to grant which have been award by the gla sinc 2013 the dataset use the 360 give standard to ensur the data is clear and access thi dataset can also be access on the 360 give navig site grantnav which allow grantmak and other to explor how grant are use area of common between grantmak and gap that are not reach by grantmak potenti grantseek can also see who and what the gla ha fund in the past

the london fire brigad attend a rang of nonfir incid which we call special servic these special servic includ assist to anim that may be trap or in distress We routin get ask for inform about the number of such incid attend by the london fire brigad and thi data is publish on the london datastor to assist those who requir it the data is provid from januari 2009 and isupd monthli A rang of inform is suppli for each incid includ some locat inform postcod borough ward as well as the datatim of the incid We do not routin record data about anim death or injuri

In [108]:
#stemmed_words = list(map(lambda x: porter_stemmer.stem(x), words_test.split()))
#stemmed_words = " ".join(stemmed_words)

In [ ]:
data_files.to_csv('./data_file_web.csv', index=True)